In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.dataset import *
from fastai.structured import *
from fastai.column_data import *

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from math import ceil

from itertools import product
import gc
from dateutil import parser

In [3]:
torch.cuda.set_device(0)

In [4]:
import fastai; fastai.__file__

'/home/paperspace/fastai/fastai/__init__.py'

In [5]:
PATH = "data/"
sz=224
arch=resnext101_64
batch_size=5

## Feature Engineering

In [11]:
item_categories = pd.read_csv(f'{PATH}item_categories.csv')
items = pd.read_csv(f'{PATH}items.csv')
shops = pd.read_csv(f'{PATH}shops.csv')
sales = pd.read_csv(f'{PATH}sales_train.csv')
sales_test = pd.read_csv(f'{PATH}test.csv')

In [174]:
l_cat = list(item_categories.item_category_name)
for ind in range(0,1):
    l_cat[ind] = 'PC Headsets / Headphones'
for ind in range(1,8):
    l_cat[ind] = 'Access'
l_cat[8] = 'Tickets (figure)'
l_cat[9] = 'Delivery of goods'
for ind in range(10,18):
    l_cat[ind] = 'Consoles'
for ind in range(18,25):
    l_cat[ind] = 'Consoles Games'
l_cat[25] = 'Accessories for games'
for ind in range(26,28):
    l_cat[ind] = 'phone games'
for ind in range(28,32):
    l_cat[ind] = 'CD games'
for ind in range(32,37):
    l_cat[ind] = 'Card'
for ind in range(37,43):
    l_cat[ind] = 'Movie'
for ind in range(43,55):
    l_cat[ind] = 'Books'
for ind in range(55,61):
    l_cat[ind] = 'Music'
for ind in range(61,73):
    l_cat[ind] = 'Gifts'
for ind in range(73,79):
    l_cat[ind] = 'Soft'
for ind in range(79,81):
    l_cat[ind] = 'Office'
for ind in range(81,83):
    l_cat[ind] = 'Clean'
l_cat[83] = 'Elements of a food'

In [ ]:
from sklearn import preprocessing
lb = preprocessing.LabelEncoder()
item_categories['item_cat_id_fix'] = lb.fit_transform(l_cat)

In [180]:
index_cols  = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'},'item_price': np.mean})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
# all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
all_data = pd.merge(grid,gb,how='left',on=index_cols)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

del grid,gb,cur_shops,cur_items
del sales
gc.collect()

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


KeyboardInterrupt: 

In [ ]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

all_data.fillna(0.0,inplace=True)
all_data['target'] = all_data['target'].clip(0.0,20.0)
all_data['month'] = all_data['date_block_num'] % 12
all_data['year'] = all_data['date_block_num'] // 12

sales_test['date_block_num'] = 34
sales_test['month'] = sales_test['date_block_num']%12
sales_test['year'] = sales_test['date_block_num']//12

all_data = pd.concat([all_data,sales_test])

del sales_test
gc.collect()

In [ ]:
all_data = all_data.merge(items, on="item_id")
all_data.drop('item_name',axis = 1,inplace=True)
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

all_data = downcast_dtypes(all_data)

del items
gc.collect()

In [ ]:
lag_variable = ['target']
lags = [1 ,2 ,3 ,4, 5, 12]
for lag in lags:
    sales_new_df = all_data[['date_block_num','shop_id','item_id']+lag_variable].copy()
    sales_new_df.date_block_num+=lag
    sales_new_df.columns = ['date_block_num','shop_id','item_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variable]
    all_data = pd.merge(all_data, sales_new_df,on=['date_block_num','shop_id','item_id'] ,how='left')
    del sales_new_df
    gc.collect()

for feat in all_data.columns:
    if 'target' in feat:
        all_data[feat]=all_data[feat].fillna(0)

In [ ]:

all_data['avg_sales_per_shop_id'] = 0.0
all_data['avg_sales_per_item_id'] = 0.0
all_data['avg_sales_per_item_cat_id'] = 0.0

avg_shop_sales_per_month = all_data.groupby(['shop_id','date_block_num'],as_index=False).agg({'target':'mean'})
avg_item_sales_per_month = all_data.groupby(['item_id','date_block_num'],as_index=False).agg({'target':'mean'})
avg_item_cat_sales_per_month = all_data.groupby(['item_category_id','date_block_num', ],as_index=False).agg({'target':'mean'})

#lets calculate  6 lags
lag_variable = ['item_id','shop_id','item_category_id']
lags = [1 ,2 ,3 ,4, 5, 12]
curr = 0
for lag in lags:
    diff = lag - curr
    curr = lag
    avg_shop_sales_per_month.date_block_num+=diff
    avg_item_cat_sales_per_month.date_block_num+=diff
    avg_item_sales_per_month.date_block_num+=diff
    
    avg_shop_sales_per_month.columns = ['shop_id','date_block_num']+ [lag_variable[1]+'_lag_'+str(lag)]
    avg_item_sales_per_month.columns = ['item_id','date_block_num']+ [lag_variable[0]+'_lag_'+str(lag)]
    avg_item_cat_sales_per_month.columns = ['item_category_id','date_block_num']+ [lag_variable[2]+'_lag_'+str(lag)]
    
    all_data = pd.merge(all_data, avg_shop_sales_per_month,on=['date_block_num','shop_id'] ,how='left')
    all_data = pd.merge(all_data, avg_item_sales_per_month,on=['date_block_num','item_id'] ,how='left')
    all_data = pd.merge(all_data, avg_item_cat_sales_per_month,on=['date_block_num','item_category_id'] ,how='left')
    
    gc.collect()
del avg_item_cat_sales_per_month,avg_item_sales_per_month,avg_shop_sales_per_month

In [ ]:
all_data.drop(columns=['mean'],inplace=True)
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [ ]:
all_data.fillna(0,inplace=True)
gc.collect()

In [124]:
all_data.columns

Index(['ID', 'date_block_num', 'item_id', 'month', 'shop_id', 'target', 'year',
       'item_category_id', 'target_lag_1', 'target_lag_2', 'target_lag_3',
       'target_lag_4', 'target_lag_5', 'target_lag_12',
       'avg_sales_per_shop_id', 'avg_sales_per_item_id',
       'avg_sales_per_item_cat_id', 'shop_id_lag_1', 'item_id_lag_1',
       'item_category_id_lag_1', 'shop_id_lag_2', 'item_id_lag_2',
       'item_category_id_lag_2', 'shop_id_lag_3', 'item_id_lag_3',
       'item_category_id_lag_3', 'shop_id_lag_4', 'item_id_lag_4',
       'item_category_id_lag_4', 'shop_id_lag_5', 'item_id_lag_5',
       'item_category_id_lag_5', 'shop_id_lag_12', 'item_id_lag_12',
       'item_category_id_lag_12'],
      dtype='object')

In [125]:
all_data.to_pickle(PATH + 'all_data.pkl')

In [181]:
all_data = pd.read_pickle(PATH + 'all_data.pkl')

In [16]:
cat_vars = [
    'date_block_num',
    'item_id',
    'month',
    'shop_id',
    'year',
    'item_category_id',
]

dep = 'target'

In [128]:
all_data.columns

Index(['ID', 'date_block_num', 'item_id', 'month', 'shop_id', 'target', 'year',
       'item_category_id', 'target_lag_1', 'target_lag_2', 'target_lag_3',
       'target_lag_4', 'target_lag_5', 'target_lag_12',
       'avg_sales_per_shop_id', 'avg_sales_per_item_id',
       'avg_sales_per_item_cat_id', 'shop_id_lag_1', 'item_id_lag_1',
       'item_category_id_lag_1', 'shop_id_lag_2', 'item_id_lag_2',
       'item_category_id_lag_2', 'shop_id_lag_3', 'item_id_lag_3',
       'item_category_id_lag_3', 'shop_id_lag_4', 'item_id_lag_4',
       'item_category_id_lag_4', 'shop_id_lag_5', 'item_id_lag_5',
       'item_category_id_lag_5', 'shop_id_lag_12', 'item_id_lag_12',
       'item_category_id_lag_12'],
      dtype='object')

In [129]:
for cat in cat_vars:
    all_data[cat] = all_data[cat].astype('category').cat.as_ordered()

In [182]:
X_train = all_data[(all_data['date_block_num']>12)&(all_data['date_block_num']<33)].drop(['target','ID'], axis = 1)
y_train = all_data[(all_data['date_block_num']>12)&(all_data['date_block_num']<33)]['target']
X_val =  all_data[all_data['date_block_num']==33].drop(['target','ID'], axis = 1)
y_val = all_data[all_data['date_block_num'] == 33]['target']
X_test = all_data[(all_data['date_block_num']==34)].drop(['target'],axis=1)
del all_data

In [183]:
X_test = X_test.drop('ID', axis=1)

In [132]:
gc.collect()

63

In [133]:
X_train.to_pickle(PATH + 'X_train.pkl')
y_train.to_pickle(PATH + 'y_train.pkl')
X_val.to_pickle(PATH + 'X_val.pkl')
y_val.to_pickle(PATH + 'y_val.pkl')
X_test.to_pickle(PATH + 'X_test.pkl')

In [134]:
X_train.head()

,date_block_num,item_id,month,shop_id,year,item_category_id,target_lag_1,target_lag_2,target_lag_3,target_lag_4,...,item_category_id_lag_3,shop_id_lag_4,item_id_lag_4,item_category_id_lag_4,shop_id_lag_5,item_id_lag_5,item_category_id_lag_5,shop_id_lag_12,item_id_lag_12,item_category_id_lag_12
4836102,13,30,1,2,1,40,0.0,0.0,0.0,0.0,...,0.244913,0.089066,0.521739,0.236217,0.107034,0.288889,0.227597,0.059868,12.456522,0.266213
4836103,13,31,1,2,1,37,0.0,0.0,0.0,0.0,...,0.183129,0.089066,0.543478,0.170779,0.107034,0.422222,0.198978,0.059868,11.195652,0.196096
4836104,13,32,1,2,1,40,1.0,0.0,0.0,0.0,...,0.244913,0.089066,1.260870,0.236217,0.107034,1.311111,0.227597,0.059868,4.478261,0.266213
4836105,13,33,1,2,1,37,1.0,1.0,2.0,0.0,...,0.183129,0.089066,0.717391,0.170779,0.107034,0.955556,0.198978,0.059868,0.847826,0.196096
4836106,13,34,1,2,1,40,0.0,0.0,0.0,0.0,...,0.244913,0.089066,0.043478,0.236217,0.107034,0.200000,0.227597,0.059868,0.239130,0.266213


In [135]:
X_test.head()

,date_block_num,item_id,month,shop_id,year,item_category_id,target_lag_1,target_lag_2,target_lag_3,target_lag_4,...,item_category_id_lag_3,shop_id_lag_4,item_id_lag_4,item_category_id_lag_4,shop_id_lag_5,item_id_lag_5,item_category_id_lag_5,shop_id_lag_12,item_id_lag_12,item_category_id_lag_12
10913850,34,30,10,2,2,40,0.0,0.0,0.0,0.0,...,0.242183,0.14334,0.093023,0.247946,0.148198,0.093023,0.249878,0.155963,0.26,0.262451
10913851,34,31,10,2,2,37,1.0,0.0,0.0,0.0,...,0.191957,0.14334,0.139535,0.156834,0.148198,0.232558,0.168300,0.155963,0.22,0.156944
10913852,34,32,10,2,2,40,0.0,0.0,1.0,0.0,...,0.242183,0.14334,0.488372,0.247946,0.148198,0.604651,0.249878,0.155963,0.58,0.262451
10913853,34,33,10,2,2,37,0.0,1.0,0.0,1.0,...,0.191957,0.14334,0.348837,0.156834,0.148198,0.255814,0.168300,0.155963,0.40,0.156944
10913854,34,38,10,2,2,41,0.0,0.0,0.0,0.0,...,0.139961,0.14334,0.116279,0.143554,0.148198,0.046512,0.113528,0.155963,0.20,0.154545


# Training

## _Deep Learning Approach_

In [6]:
X_train = pd.read_pickle(PATH + 'X_train.pkl')
y_train = pd.read_pickle(PATH + 'y_train.pkl')
X_val = pd.read_pickle(PATH + 'X_val.pkl')
y_val = pd.read_pickle(PATH + 'y_val.pkl')
X_test = pd.read_pickle(PATH + 'X_test.pkl')

In [17]:
X_test = X_test.merge(sales_test, on=['item_id', 'shop_id'], how='inner')
X_test = X_test.set_index('ID').sort_index().reset_index().drop('ID', axis=1)

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


In [20]:
def rmse(y_pred, targ):
    return math.sqrt(((targ - y_pred)**2).mean())

In [21]:
X_train['target'] = y_train
X_val['target'] = y_val
X_test['target'] = 0

In [22]:
df, y, nas, mapper = proc_df(X_train, 'target', do_scale=True)
val_df, val_y, nas, mapper = proc_df(
    X_val,
    dep,
    do_scale=True,
    mapper=mapper,
)
test_df, _, nas, mapper = proc_df(
    X_test,
    dep,
    do_scale=True,
    mapper=mapper,
)

In [23]:
test_df.columns

Index(['date_block_num', 'item_id', 'month', 'shop_id', 'year',
       'item_category_id', 'target_lag_1', 'target_lag_2', 'target_lag_3',
       'target_lag_4', 'target_lag_5', 'target_lag_12',
       'avg_sales_per_shop_id', 'avg_sales_per_item_id',
       'avg_sales_per_item_cat_id', 'shop_id_lag_1', 'item_id_lag_1',
       'item_category_id_lag_1', 'shop_id_lag_2', 'item_id_lag_2',
       'item_category_id_lag_2', 'shop_id_lag_3', 'item_id_lag_3',
       'item_category_id_lag_3', 'shop_id_lag_4', 'item_id_lag_4',
       'item_category_id_lag_4', 'shop_id_lag_5', 'item_id_lag_5',
       'item_category_id_lag_5', 'shop_id_lag_12', 'item_id_lag_12',
       'item_category_id_lag_12'],
      dtype='object')

In [24]:
y_train = (y.clip(0.,40.))
y_val = (val_y.clip(0.,40.))

In [25]:
md = ColumnarModelData.from_data_frames(
    path=PATH,
    trn_df=df,
    trn_y=y_train,
    val_df=val_df,
    val_y=y_val,
    cat_flds=cat_vars,
    bs=256,
    test_df=test_df,
    is_reg=True,  # is regression
    is_multi=False, 
)

In [26]:
cat_sz = [(c, len(X_train[c].cat.categories)+1) for c in cat_vars]

In [27]:
cat_sz

[('date_block_num', 36),
 ('item_id', 22171),
 ('month', 13),
 ('shop_id', 61),
 ('year', 4),
 ('item_category_id', 85)]

In [28]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [29]:
emb_szs

[(36, 18), (22171, 50), (13, 7), (61, 31), (4, 2), (85, 43)]

In [33]:
m = md.get_learner(
    emb_szs=emb_szs,
    n_cont=len(df.columns) - len(cat_vars),
    emb_drop=.01,
    out_sz=1,
    szs=[1000, 500, 500],
    drops=[.001, .005, .005],
    use_bn=True,
)

/home/paperspace/fastai/fastai/column_data.py:101: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  for o in self.lins: kaiming_normal(o.weight.data)
/home/paperspace/fastai/fastai/column_data.py:103: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  kaiming_normal(self.outp.weight.data)


In [34]:
m.crit = nn.MSELoss()

In [32]:
m.lr_find()

  0%|          | 0/22811 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
m.sched.plot()

In [35]:
lr = 1e-5
m.fit(lrs=lr, n_cycle=2, metrics=[rmse], cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss   rmse                             
    0      0.912345   0.90044    0.791529  
    1      0.734885   0.870398   0.783333                         
    2      0.734734   0.867583   0.785855                         



[0.86758313364853312, 0.78585454272984123]

### Random Forest Regressor

In [42]:
val_y

array([0., 1., 0., ..., 0., 0., 0.], dtype=float32)

In [43]:
y

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [28]:
rf_m = RandomForestRegressor(n_jobs=-1)

In [46]:
rf_m.fit(df, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [47]:
result = rf_m.predict(val_df)
rmse(result, val_y)

1.064909110803377

In [150]:
rf_result = rf_m.predict(test_df)

In [ ]:
np.unique(rf_result)

In [ ]:
predictions

## Gradient Boost Regressor

In [184]:
sales_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [185]:
X_test.head()

,date_block_num,item_id,month,shop_id,year,item_category_id,target_lag_1,target_lag_2,target_lag_3,target_lag_4,...,item_category_id_lag_3,shop_id_lag_4,item_id_lag_4,item_category_id_lag_4,shop_id_lag_5,item_id_lag_5,item_category_id_lag_5,shop_id_lag_12,item_id_lag_12,item_category_id_lag_12
10913850,34,30,10,2,2,40,0.0,0.0,0.0,0.0,...,0.242183,0.14334,0.093023,0.247946,0.148198,0.093023,0.249878,0.155963,0.26,0.262451
10913851,34,31,10,2,2,37,1.0,0.0,0.0,0.0,...,0.191957,0.14334,0.139535,0.156834,0.148198,0.232558,0.168300,0.155963,0.22,0.156944
10913852,34,32,10,2,2,40,0.0,0.0,1.0,0.0,...,0.242183,0.14334,0.488372,0.247946,0.148198,0.604651,0.249878,0.155963,0.58,0.262451
10913853,34,33,10,2,2,37,0.0,1.0,0.0,1.0,...,0.191957,0.14334,0.348837,0.156834,0.148198,0.255814,0.168300,0.155963,0.40,0.156944
10913854,34,38,10,2,2,41,0.0,0.0,0.0,0.0,...,0.139961,0.14334,0.116279,0.143554,0.148198,0.046512,0.113528,0.155963,0.20,0.154545


In [191]:
asdf = X_test.merge(sales_test, on=['item_id', 'shop_id'], how='inner')

In [192]:
len(asdf)

214200

In [198]:
test_asdf = asdf.set_index('ID').sort_index().reset_index().drop('ID', axis=1)

In [148]:
for var in cat_vars:
    X_train[var] = X_train[var].astype('int32')
    X_val[var] = X_val[var].astype('int32')
    X_test[var] = X_test[var].astype('int32')

In [149]:
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
params = {
    'objective': 'reg:linear',
    'max_depth':4 ,
    'learning_rate': 1,
    'silent': 1.0,
    'n_estimators': 30,
    'min_child_weight':10
}

bst = XGBRegressor(**params).fit(X_train, y_train,eval_metric='rmse')
preds = bst.predict(X_val)
sqrt(mean_squared_error(y_val, preds))

0.9399399477464334

In [141]:
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
params = {
    'objective': 'reg:linear',
    'max_depth':4 ,
    'learning_rate': 1,
    'silent': 1.0,
    'n_estimators': 30,
    'min_child_weight':10
}

X = pd.concat([X_train,X_val])
y = np.concatenate([y_train,y_val])
del X_train,X_val,y_train,y_val
gc.collect()

373

In [142]:
bst = XGBRegressor(**params).fit(X, y,eval_metric='rmse')

In [151]:
preds = bst.predict(X_test)
preds = np.clip(preds,0.0,20.0)

In [202]:
preds = bst.predict(test_asdf)
preds = np.clip(preds, 0.0,20.0)

In [204]:
preds

array([0.65605, 0.25851, 1.01994, ..., 0.04138, 0.00529, 0.01926], dtype=float32)

## Submit It!

In [36]:
result = m.predict(is_test=True)
# predictions = np.expm1(result).clip(0, 20)
predictions = result.clip(0,20)
display(predictions)

array([[ 0.63443],
       [ 1.00198],
       [ 0.49965],
       ..., 
       [ 0.19427],
       [ 0.03526],
       [ 0.4749 ]], dtype=float32)

In [37]:
final_dataframe = pd.read_csv(f'{PATH}test.csv')

In [38]:
final_dataframe['item_cnt_month'] = predictions
final_dataframe = final_dataframe[['ID', 'item_cnt_month']]

In [39]:
final_dataframe.head()

,ID,item_cnt_month
0,0,0.634432
1,1,1.001979
2,2,0.499649
3,3,0.099903
4,4,1.990264


In [40]:
len(final_dataframe)

214200

In [41]:
len(np.unique(final_dataframe['ID']))

214200

In [42]:
SUBM = 'sub/'
os.makedirs(SUBM, exist_ok=True)
final_dataframe.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [43]:
FileLink(f'{SUBM}subm.gz')

/home/paperspace/kaggle/competitive-data-science-predict-future-sales/sub/subm.gz